## Query Code - Location of closest coordinate out of a list of coordinates. 

##### - https://www.timvink.nl/closest-coordinates/
##### - https://stackoverflow.com/questions/39107896/efficiently-finding-the-closest-coordinate-pair-from-a-set-in-python

In [77]:
import pandas as pd
from geopy.distance import great_circle
import math
from scipy import spatial

from scipy import spatial

places = []
for index, row in geonames.iterrows():
    coordinates = [row['latitude'], row['longitude']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

tree = spatial.KDTree(places)

def find_population(lat, lon):
    cartesian_coord = cartesian(lat, lon)
    closest = tree.query([cartesian_coord], p = 2)
    index = closest[1][0]
    return {
        'name' : geonames.name[index],
        'latitude' : geonames.latitude[index],
        'longitude' : geonames.longitude[index],
        'population' : geonames.population[index],
        'distance' : closest[0][0]
    }

# Load 1.88M Fire Database and Sample Emissions Cluster Year

In [78]:
usdafiredb_onemil = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/1_188m_USDA Fire Database/Fire_Program_Analysis__Fire_Occurrence_Database_Feature_Layer.csv')

In [79]:
FiresClusters2011 = pd.read_csv('/Users/AlfHaugen/Python/Wildfire_Data/FireExports/Emissions Cluster Data/Emissions2011_DBScan_Clusters.csv')

In [80]:
pd.set_option('display.max_columns', 40)

In [81]:
usdafiredb_onemil.shape

(1880465, 40)

In [82]:
usdafiredb_onemil.head(3)

,X,Y,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,-111.350290,44.495840,2001,2016,FS-1423220,FED,FS-FIRESTAT,FS,USIDCTF,Caribou-Targhee National Forest,415,Caribou-Targhee National Forest,28,52-07,B44P,COFFEE POT,NaN,NaN,NaN,NaN,NaN,2005,2005-08-25T00:00:00.000Z,237,1400.0,4,Campfire,2005-08-25T00:00:00.000Z,237.0,1440.0,0.10,A,44.495833,-111.350278,5,USFS,ID,43,43.0,Fremont
1,-110.567235,46.660562,2002,2017,FS-1423221,FED,FS-FIRESTAT,FS,USMTLCF,Lewis & Clark National Forest,115,Lewis and Clark National Forest,3,MT-LCF-013,BEW2,LION CREEK,NaN,NaN,NaN,NaN,NaN,2005,2005-07-14T00:00:00.000Z,195,830.0,1,Lightning,2005-07-16T00:00:00.000Z,197.0,1419.0,0.10,A,46.660556,-110.567222,5,USFS,MT,59,59.0,Meagher
2,-114.475013,44.798062,2003,2018,FS-1423222,FED,FS-FIRESTAT,FS,USIDSCF,Salmon-Challis National Forest,413,Salmon-Challis National Forest,57,0613,BV2X,CASTLE,NaN,NaN,NaN,NaN,NaN,2005,2005-08-09T00:00:00.000Z,221,1140.0,1,Lightning,2005-08-10T00:00:00.000Z,222.0,1200.0,0.25,A,44.798056,-114.475000,5,USFS,ID,59,59.0,Lemhi


In [83]:
FiresClusters2011.head(3)

,Unnamed: 0,id,year,doy,longitude,latitude,grid10k,covertype,fuelcode,area_burned,prefire_fuel,consumed_fuel,ECO2,ECO,ECH4,EPM2.5,cwd_frac,duff_frac,fuel_moisture_class,burn_source,burnday_source,BSEV,BSEV_flag
0,0,239.0,2011.0,97.0,-81.0521,25.2176,4536.0,3.0,1600.0,0.0,6220.097576,1999.757720,3347.594423,153.981344,4.499455,23.797117,0.023231,0.082115,3.0,1.0,15.0,1.0,0.0
1,1,581.0,2011.0,2.0,-97.7979,26.7378,6671.0,1.0,1.0,62500.0,293.889492,273.317228,458.352991,19.132206,0.732490,2.787836,0.000000,0.000000,2.0,1.0,15.0,2.0,0.0
2,2,728.0,2011.0,163.0,-98.3282,26.7773,6666.0,1.0,1.0,62500.0,175.414590,163.135569,273.578349,11.419490,0.437203,1.663983,0.000000,0.000000,2.0,1.0,78.0,2.0,0.0


In [84]:
FiresClusters2011.shape

(10554, 23)

# Settting up Sample Data for the year 2011

### Pairing Down Cluster Data to select a single sample - Temp. 
pd.set_option('display.max_columns', 30)
FCECO2011 = FiresClusters2011[(FiresClusters2011.ECO > 800)]
FCECO2011.head(5)

FCECO2011.shape

In [88]:
### Creating smaller DF for 2011 Unique Fires from USDA 1.88m fire set. 
usdafires2011 = usdafiredb_onemil[(usdafiredb_onemil.FIRE_YEAR == 2011)]
usdafires2011 = usdafires2011.drop(['SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 
                                    'LOCAL_INCIDENT_ID', 'OWNER_CODE','OWNER_DESCR', 'X', 'Y','FOD_ID', 'FPA_ID', 
                                    'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM'], 1)
usdafires2011.shape

(90552, 28)

In [89]:
usdafires2011.head(3)

,OBJECTID,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
1453964,1453965,FS,USAKCGF,Chugach National Forest,EK3T,VFD BEAR CREEK #1,NaN,NaN,NaN,NaN,NaN,2011,2011-03-21T00:00:00.000Z,80,1212.0,6,Railroad,2011-03-21T00:00:00.000Z,80.0,1618.0,0.1,A,60.246389,-149.349444,AK,NaN,NaN,NaN
1453965,1453966,FS,USAKCGF,Chugach National Forest,F72E,CPR LNDG ORGANIC DMP,NaN,NaN,NaN,NaN,NaN,2011,2011-04-15T00:00:00.000Z,105,1812.0,5,Debris Burning,2011-04-18T00:00:00.000Z,108.0,1156.0,0.1,A,60.475833,-149.752500,AK,NaN,NaN,NaN
1453966,1453967,FS,USAKCGF,Chugach National Forest,EK7P,TOKLAT WAY DEBRIS,NaN,NaN,NaN,NaN,NaN,2011,2011-05-11T00:00:00.000Z,131,1250.0,5,Debris Burning,2011-05-11T00:00:00.000Z,131.0,1331.0,0.1,A,60.514444,-149.467500,AK,NaN,NaN,NaN


## Using Nearest Neighboars/KD Tree to build function to find nearest coordinate out of a data set. 

In [90]:
def cartesian(latitude, longitude, elevation = 0):
    # Convert to radians
    latitude = latitude * (math.pi / 180)
    longitude = longitude * (math.pi / 180)

    R = 6371 # 6378137.0 + elevation  # relative to centre of the earth
    X = R * math.cos(latitude) * math.cos(longitude)
    Y = R * math.cos(latitude) * math.sin(longitude)
    Z = R * math.sin(latitude)
    return (X, Y, Z)

In [91]:
places = []
for index, row in usdafires2011.iterrows():
    coordinates = [row['LATITUDE'], row['LONGITUDE']]
    cartesian_coord = cartesian(*coordinates)
    places.append(cartesian_coord)

tree = spatial.KDTree(places)
places[1]

(-2712.1402271451984, -1581.5148494831365, 5543.71239957171)

In [92]:
def find_firecluster(lat, lon):
    cartesian_coord = cartesian(lat, lon)
    closest = tree.query([cartesian_coord], p = 2)
    index = closest[1][0]
    return closest

In [14]:
targetlat = (38.4782)
targetlog = (-120.3440)

resultfire0 = find_firecluster(targetlat, targetlog)
resultfire0

(array([6.23206418]), array([59819]))

In [15]:
resultfire = find_firecluster(38.4782, -120.3440)
resultfire

(array([6.23206418]), array([59819]))

## Build new DataFrame to hold selection data. 

In [41]:
clu1 = pd.DataFrame({'clusterlat': [targetlat], 'clusterlog': [targetlog], 'distance': resultfire[0], 'resultrow': resultfire[1]})
#clu = clue.assign('clusterlat': targetlat, 'clusterlog': targetlog)
indexnum = resultfire[1]
clu1

,clusterlat,clusterlog,distance,resultrow
0,38.4782,-120.344,6.232064,59819


In [51]:
### Testing to see how to pull out number from DF, to use as input: 
indexnum = clu1.loc[0]['resultrow']
indexnum = int(indexnum)
indexnum

59819

usdafires2011.iloc[indexnum]

In [57]:
unique_lat = usdafires2011.iloc[indexnum]['LATITUDE']
unique_log = usdafires2011.iloc[indexnum]['LONGITUDE']
unique_firename = usdafires2011.iloc[indexnum]['FIRE_NAME']
unique_firesize = usdafires2011.iloc[indexnum]['FIRE_SIZE']
unique_fireclass = usdafires2011.iloc[indexnum]['FIRE_SIZE_CLASS']
unique_discdate = usdafires2011.iloc[indexnum]['DISCOVERY_DOY']
unique_contdate = usdafires2011.iloc[indexnum]['CONT_DOY']

In [64]:
clu2 = clu1.assign(unique_lat1 = unique_lat, unique_log = unique_log, unique_firename = unique_firename,unique_firesize = unique_firesize, unique_fireclass = unique_fireclass,unique_discdate = unique_discdate, unique_contdate = unique_contdate)

In [65]:
clu2

,clusterlat,clusterlog,distance,resultrow,unique_lat,unique_lat1,unique_log,unique_firename,unique_firesize,unique_fireclass,unique_discdate,unique_contdate
0,38.4782,-120.344,6.232064,59819,38.445,38.445,-120.401667,EAST,44.0,C,335,NaN


# Function to locate Closest Coordinates and Pull data features from target DF to main DF. 

In [93]:
 ### clusterdf will be panda dataframe that has cluster coordinates to cross-ref with the     
def findclustermatch(clusterdf, targetdf):
    clustertotargetdf = pd.DataFrame()
    clustindexnum = 0
    clustindexnum = int(clustindexnum)
    while clustindexnum < len(clusterdf):
        #capture data from target/cluster to then use to find target match in other dataframe:
        clusterid = clusterdf.iloc[clustindexnum]['id']
        clusteryear = clusterdf.iloc[clustindexnum]['year']
        clusterdoy = clusterdf.iloc[clustindexnum]['doy']
        clusterlat = clusterdf.iloc[clustindexnum]['latitude']
        clusterlong = clusterdf.iloc[clustindexnum]['longitude']
        
        # Running cluster find function:
        distance_location = find_firecluster(clusterlat, clusterlong)
        targetlocation = distance_location[1]
        targetlocation = int(targetlocation)
        
        #Lines to pull data from the target dataframe, will need to be customized to the target DF. 
        target_lat = targetdf.iloc[targetlocation]['LATITUDE']
        target_long = targetdf.iloc[targetlocation]['LONGITUDE']
        target_firename = targetdf.iloc[targetlocation]['FIRE_NAME']
        target_firesize = targetdf.iloc[targetlocation]['FIRE_SIZE']
        target_fireclass = targetdf.iloc[targetlocation]['FIRE_SIZE_CLASS']
        target_year = targetdf.iloc[targetlocation]['FIRE_YEAR']
        target_discdoy = targetdf.iloc[targetlocation]['DISCOVERY_DOY']
        target_contdoy = targetdf.iloc[targetlocation]['CONT_DOY']
        
        #adfsd
        cdftemp = pd.DataFrame({'cluster_index':[clustindexnum], 'cluster_id': [clusterid], 'clusteryear':[clusteryear], 'clusterdoy':[clusterdoy], 'clusterlat': [clusterlat], 'clusterlog': [clusterlong], 
                                'distance': distance_location[0], 'resultrow': distance_location[1], 'targetlat':[target_lat], 'targetlong':[target_long], 'firename':[target_firename],
                                'firesize':[target_firesize], 'fireclass':[target_fireclass],'fire_year':[target_year], 'discovery_doy': [target_discdoy], 'contain_doy': [target_contdoy]})
        
        clustertotargetdf = clustertotargetdf.append(cdftemp, ignore_index = True)
        clustindexnum = clustindexnum + 1
    return clustertotargetdf

In [95]:
cluster_targetmatch = findclustermatch(FCECO2011,usdafires2011)
cluster_targetmatch.tail()


,cluster_index,cluster_id,clusteryear,clusterdoy,clusterlat,clusterlog,distance,resultrow,targetlat,targetlong,firename,firesize,fireclass,fire_year,discovery_doy,contain_doy
9,9,900090.0,2011.0,121.0,44.5347,-85.0806,0.110544,46636,44.534340,-85.081900,KALKASKA 02 BEAVER DAM ...,13.6,C,2011,121,121.0
10,10,900947.0,2011.0,281.0,46.6406,-94.2801,0.057481,46461,46.640470,-94.279371,NaN,11.0,C,2011,280,280.0
11,11,901138.0,2011.0,234.0,46.8194,-87.8670,0.152388,46304,46.818091,-87.867593,WINDY MOUNTAIN ...,29.2,C,2011,234,242.0
12,12,901521.0,2011.0,238.0,47.7328,-91.4668,0.041867,288,47.733056,-91.466389,LITTLE BEAR,14.0,C,2011,238,240.0
13,13,901541.0,2011.0,281.0,47.8303,-95.3753,0.088631,9367,47.830000,-95.374200,RED LAKE 306,420.0,E,2011,279,283.0


In [96]:
cluster_targetmatch = findclustermatch(FiresClusters2011,usdafires2011)
cluster_targetmatch.tail(20)

,cluster_index,cluster_id,clusteryear,clusterdoy,clusterlat,clusterlog,distance,resultrow,targetlat,targetlong,firename,firesize,fireclass,fire_year,discovery_doy,contain_doy
10534,10534,902348.0,2011.0,218.0,47.7107,-117.6217,0.109254,18621,47.710150,-117.622910,RAMBO,18.5,C,2011,218,219.0
10535,10535,902358.0,2011.0,254.0,48.1932,-115.3735,0.111742,41274,48.193300,-115.375000,HARRIS,16.0,C,2011,254,255.0
10536,10536,902360.0,2011.0,205.0,48.0312,-116.9668,0.124721,48048,48.030082,-116.966665,HUNTER FIRE,16.5,C,2011,205,206.0
10537,10537,902381.0,2011.0,224.0,47.6498,-119.3722,0.089679,15498,47.649200,-119.373000,HIGHWAY 17,600.0,E,2011,224,225.0
10538,10538,902405.0,2011.0,208.0,48.6762,-112.6797,0.122367,10336,48.675120,-112.679380,3 MILE FIRE,80.0,C,2011,208,208.0
10539,10539,902409.0,2011.0,99.0,48.1773,-118.0817,0.117962,18743,48.177930,-118.080420,SOUTH HARVEY CREEK,15.0,C,2011,99,99.0
10540,10540,902410.0,2011.0,303.0,48.0730,-118.7330,0.026079,9908,48.073110,-118.732690,MEADOW CR POWERLINE,23.7,C,2011,303,304.0
10541,10541,902411.0,2011.0,229.0,47.2797,-123.2388,0.049401,18550,47.280134,-123.238940,4 CORNERS RD,1.1,B,2011,229,230.0
10542,10542,902415.0,2011.0,234.0,48.7433,-114.7117,0.086039,41127,48.742600,-114.711200,FITZSIMMONS,22.5,C,2011,234,244.0
10543,10543,902417.0,2011.0,308.0,48.0346,-119.6339,0.213191,9909,48.034810,-119.636750,LAGRANGE ROAD,62.0,C,2011,308,309.0


In [98]:
cluster_targetmatch.head(100)

,cluster_index,cluster_id,clusteryear,clusterdoy,clusterlat,clusterlog,distance,resultrow,targetlat,targetlong,firename,firesize,fireclass,fire_year,discovery_doy,contain_doy
0,0,239.0,2011.0,97.0,25.2176,-81.0521,28.322325,16374,25.327140,-80.797810,SWEETBAY POND,0.1,A,2011,92,93.0
1,1,581.0,2011.0,2.0,26.7378,-97.7979,0.982374,16637,26.729167,-97.800000,KING RANCH,8000.0,G,2011,2,4.0
2,2,728.0,2011.0,163.0,26.7773,-98.3282,4.876773,68652,26.785833,-98.376389,ENCINITOS FIRE,3200.0,F,2011,163,165.0
3,3,1714.0,2011.0,159.0,25.7770,-80.6211,3.232477,27837,25.760400,-80.594600,MINI MICCOSUKEE ( 6 ),0.1,A,2011,168,168.0
4,4,3494.0,2011.0,159.0,25.8300,-80.5742,8.004116,27837,25.760400,-80.594600,MINI MICCOSUKEE ( 6 ),0.1,A,2011,168,168.0
5,5,4772.0,2011.0,38.0,25.9641,-81.2643,4.121732,16361,25.993300,-81.289700,WAGON SOUTH,8.0,B,2011,203,205.0
6,6,2974.0,2011.0,158.0,25.7995,-80.4645,0.558434,16381,25.801900,-80.469400,DOF PRARIE FIRE,68295.0,G,2011,157,166.0
7,7,3426.0,2011.0,158.0,25.8178,-80.5088,4.322184,16381,25.801900,-80.469400,DOF PRARIE FIRE,68295.0,G,2011,157,166.0
8,8,4757.0,2011.0,51.0,25.9784,-81.3645,5.140049,16347,25.939400,-81.336900,COPELAND,110.0,D,2011,132,133.0
9,9,4997.0,2011.0,161.0,25.8667,-80.5576,4.150817,30382,25.903100,-80.566800,#1 SEASON FIRE,600.0,E,2011,19,20.0
